First, we need a 'chessboard' to hold numbers.

In [1]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
# 生成随机棋盘
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = []
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [ ]:
n = 3
random_board = generate_puzzle(n)

print("random initial board:")
for row in random_board:
    print(row)

random initial board:
[5, 6, 8]
[0, 1, 3]
[4, 7, 2]


Now, get a 3x3 'chessboard',then do something to solve this problem

In [ ]:
PUZZLE_DIM = 5
TARGET_STATE = tuple(range(1, PUZZLE_DIM**2) + [0])  # 目标状态 (1, 2, 3, 4, ..., 0)
action = NamedTuple('Action', ['pos1', 'pos2'])